In [182]:
import os
import pandas as pd
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv())
import psycopg2
import time
from sqlalchemy import create_engine
import json
from unidecode import unidecode
import re
import numpy as np
from datetime import datetime

In [183]:
def connect_db():
    try:
        alchemy_url = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
            user=os.getenv('login_sql_tech'),
            password=os.getenv('senha_sql_tech'),
            host=os.getenv('host_sql_tech'),
            port=os.getenv('port_sql_tech'),
            database=os.getenv('database_sql_tech')
        )
        return create_engine(alchemy_url)
    except:
        print("Erro ao conectar ao banco de dados")
        return False

In [184]:
engine = connect_db()

In [185]:
dataFrame = pd.read_sql_query(
    "SELECT * FROM clientes",
    con = engine
    )

In [186]:
dataFrame.head()

,id_contrato,nome,codigo_cliente,documento,sexo,estado_civil,data_nascimento,tipo,endereco,data_contrato,...,valor_contrato,valor_entrada,valor_pago_parcelas,data_atualizacao,valor_encargos,valor_iptu,codigo_unidade,id_emp,cep,score
0,1324.0,ADRIANA CONEJO,2116.0,052.961.739-02,F,Solteiro(a),1986-07-28,Física,"PAULO ROBERTO ANASTACIO ,377,PARANAGUAMIRIM ,J...",2019-10-23,...,88683.53,0.00,0.00,2022-08-03,5250.62,0.00,1591.0,blv,89.231-795,9999.00
1,4173.0,JOHNSON MAKENRO JULIEN,51121.0,800.582.619-23,M,Casado(a),1989-07-07,Física,"RUA 448,650,MORRETES,ITAPEMASC",2021-12-06,...,195900.00,19590.00,13065.29,2022-08-03,0.00,0.00,4335.0,cmv,88.220-000,995.83
2,2125.0,JOSIANE ARTUSO DIAS,50717.0,010.327.499-54,F,Casado(a),1987-12-17,Física,"DOM ALVARO NUNES CABEZA DE VACA,485,BATEL,GUAR...",2020-08-17,...,65900.00,3000.00,20849.81,2022-08-03,0.00,1000.00,2779.0,crj_orb,85.015-310,1000.00
3,335.0,ULISSES FERNANDO ZOLLNER,903.0,010.146.079-14,M,Divorciado(a),1987-02-24,Física,"Gilmar Cesar Piekarski,30,Tatuquara,CURITIBAPR",2018-02-01,...,64592.88,3391.08,35314.23,2022-08-03,319.21,874.49,389.0,mar,81.470-035,757.11
4,4544.0,MICHEL RODRIGO DOS SANTOS,51695.0,387.037.838-71,M,Casado(a),1992-04-30,Física,"MATUSALEM GONÇALVES PINHEIRO,83,PARANAGUAMIRIM...",2022-06-06,...,74900.00,4165.94,0.00,2022-08-03,0.00,0.00,5631.0,jcr,89.231-603,1000.00


In [187]:
dataFrame.drop(
    [
    'nome',
    'codigo_cliente',
    'endereco',
    'codigo_empreendimento',
    'nome_empreendimento',
    'imobiliaria',
    'lote',
    'quadra',
    'nome_spe',
    'telefone',
    'documento',
    'data_contrato',
    'data_status',
    'ltv',
    'matricula',
    'rg',
    'cep',
    'nacionalidade',
    'id_emp',
    'area',
    'valor_iptu',
    'data_atualizacao',
    'numero_aditivos',
    'codigo_unidade',
    'valor_encargos',
    'valor_entrada',
    'valor_pago_parcelas',
    ], 
    axis=1, 
    inplace=True)

dataFrame

,id_contrato,sexo,estado_civil,data_nascimento,tipo,inadimplente,status,tier,valor_contrato,score
0,1324.0,F,Solteiro(a),1986-07-28,Física,False,Distratado,,88683.53,9999.00
1,4173.0,M,Casado(a),1989-07-07,Física,False,Ativo,B,195900.00,995.83
2,2125.0,F,Casado(a),1987-12-17,Física,False,Ativo,B+,65900.00,1000.00
3,335.0,M,Divorciado(a),1987-02-24,Física,False,Ativo,C,64592.88,757.11
4,4544.0,M,Casado(a),1992-04-30,Física,False,Ativo,C,74900.00,1000.00
...,...,...,...,...,...,...,...,...,...,...
3958,739.0,M,Casado(a),1979-03-26,Física,False,Distratado,,65310.11,9999.00
3959,1655.0,M,Solteiro(a),1996-05-17,Física,False,Distratado,,72900.00,9999.00
3960,4161.0,M,Solteiro(a),1981-12-11,Física,True,Ativo,D,67900.00,311.11
3961,4347.0,M,Solteiro(a),1996-04-27,Física,False,Ativo,C,67900.00,787.04


In [188]:
lista_idade = []
for item in dataFrame.data_nascimento:
    lista_idade.append(datetime.now().year - item.year)
dataFrame['idade'] = lista_idade
dataFrame.drop(['data_nascimento'], axis=1, inplace=True)
dataFrame

,id_contrato,sexo,estado_civil,tipo,inadimplente,status,tier,valor_contrato,score,idade
0,1324.0,F,Solteiro(a),Física,False,Distratado,,88683.53,9999.00,36
1,4173.0,M,Casado(a),Física,False,Ativo,B,195900.00,995.83,33
2,2125.0,F,Casado(a),Física,False,Ativo,B+,65900.00,1000.00,35
3,335.0,M,Divorciado(a),Física,False,Ativo,C,64592.88,757.11,35
4,4544.0,M,Casado(a),Física,False,Ativo,C,74900.00,1000.00,30
...,...,...,...,...,...,...,...,...,...,...
3958,739.0,M,Casado(a),Física,False,Distratado,,65310.11,9999.00,43
3959,1655.0,M,Solteiro(a),Física,False,Distratado,,72900.00,9999.00,26
3960,4161.0,M,Solteiro(a),Física,True,Ativo,D,67900.00,311.11,41
3961,4347.0,M,Solteiro(a),Física,False,Ativo,C,67900.00,787.04,26


In [189]:
# dataFrame.tipo.unique()
# dataFrame.estado_civil.unique()
# dataFrame.estado_civil.value_counts()
# dataFrame.tier.value_counts()

In [190]:
#tratando os valores
dict_sexo = {
    'M' : 1,
    'F' : 2
}
#'Furos' na coluna tier estão sendo preenchidas com o valor mais recorrente
dict_tier = {
    '   ' : 6,
    'A  ': 1,
    'A+ ': 2,
    'B  ': 3,
    'B+ ': 4,
    'C  ': 5,
    'D  ' : 6,
    'D+ ' : 7,
    'E  ' : 8,
    'F  ' : 9,
    'S  ' : 10
}
#'Furos' na coluna estado_civil estão sendo preenchidas com o valor mais recorrente
dict_estado_civil = {
    '' : 1,
    'Solteiro(a)' : 1,
    'Casado(a)' : 2,
    'Divorciado(a)' : 3,
    'Separado(a) Judicialmente' : 4,
    'Viúvo(a)' : 5,
    'União Estável' : 6
}
dict_tipo = {
    'Física' : 1,
    'Jurídica' : 2
}
dict_inadimplente = {
    False : 1,
    True : 2
}
dict_status = {
    'Ativo' : 1,
    'Quitado' : 1,
    'Distratado' : 2
}


dataFrame.sexo = dataFrame.sexo.map(dict_sexo)
dataFrame.tier = dataFrame.tier.map(dict_tier)
dataFrame.estado_civil = dataFrame.estado_civil.map(dict_estado_civil)
dataFrame.tipo = dataFrame.tipo.map(dict_tipo)
dataFrame.inadimplente = dataFrame.inadimplente.map(dict_inadimplente)
dataFrame.status = dataFrame.status.map(dict_status)

In [191]:
dataFrame

,id_contrato,sexo,estado_civil,tipo,inadimplente,status,tier,valor_contrato,score,idade
0,1324.0,2.0,1,1,1,2,6,88683.53,9999.00,36
1,4173.0,1.0,2,1,1,1,3,195900.00,995.83,33
2,2125.0,2.0,2,1,1,1,4,65900.00,1000.00,35
3,335.0,1.0,3,1,1,1,5,64592.88,757.11,35
4,4544.0,1.0,2,1,1,1,5,74900.00,1000.00,30
...,...,...,...,...,...,...,...,...,...,...
3958,739.0,1.0,2,1,1,2,6,65310.11,9999.00,43
3959,1655.0,1.0,1,1,1,2,6,72900.00,9999.00,26
3960,4161.0,1.0,1,1,2,1,6,67900.00,311.11,41
3961,4347.0,1.0,1,1,1,1,5,67900.00,787.04,26


In [192]:
dataFrame.status.value_counts()

1    3019
2     944
Name: status, dtype: int64

In [193]:
lista_razao_pago_total = []
lista_dias_atraso_total = []
lista_parcelas = []
lista_parcelas_atraso = []


In [194]:
for row_db1 in dataFrame.itertuples():
    dataFrame_parcelas = pd.read_sql_query(
        "SELECT * FROM parcelas WHERE id_contrato = {} and (tipo = 'Parcela' or tipo = 'Entrada') ORDER BY sequencia ASC".format(row_db1.id_contrato),
    con = engine
    )

    valor_contrato = row_db1.valor_contrato

    total_dias = 0
    valor_pago = 0
    total_parcelas_pagas_com_atraso = 0
    total_parcelas_ = 0
    razao_pago_total = 0
    total_parcelas = 0

    for row in dataFrame_parcelas.itertuples():
        # print(type(row.vencimento))
        total_dias = total_dias + row.dias_atraso
        valor_pago = valor_pago + row.valor_pago
        if(row.dias_atraso > 0):
            total_parcelas_pagas_com_atraso = total_parcelas_pagas_com_atraso + 1
        if(row.vencimento > datetime.date(datetime.now())):
            break
        if(row.sequencia.split('/')[0] == '012'):
            break
        total_parcelas = total_parcelas + 1
    
    lista_razao_pago_total.append(valor_pago / valor_contrato)
    lista_dias_atraso_total.append(total_dias)
    lista_parcelas_atraso.append(total_parcelas_pagas_com_atraso)
    lista_parcelas.append(total_parcelas)


    


In [195]:
lista_razao_pago_total

[0.0,
 0.16669367023991838,
 0.17849772382397572,
 0.4134841177541548,
 0.05562002670226969,
 0.126124791086351,
 0.13697416752774905,
 0.22583736383442257,
 0.0877274336283186,
 0.10846047430830041,
 0.27142386495925486,
 0.0,
 0.18003589835877895,
 0.016869581749049428,
 0.017030398069963815,
 0.02063983488132095,
 0.15411710056151096,
 1.0,
 0.07535690200210748,
 0.10394295719027709,
 0.3127386174016687,
 0.1999063169896227,
 0.04202750185322461,
 0.016666666666666666,
 0.10080716723549489,
 0.15911497165721908,
 0.18665673420738976,
 0.18197973462002412,
 1.0,
 0.18634972677595626,
 0.844138598442714,
 0.0,
 1.0,
 0.18977082002129925,
 0.10809265442404005,
 1.0147378859525562,
 0.08132335928809788,
 0.19802370916754472,
 0.22538686679174474,
 0.0,
 0.14171580958664962,
 0.17429903978052122,
 1.0,
 1.0,
 0.10437096134786916,
 0.19875971223021577,
 0.1856103739445115,
 0.1560431855500821,
 0.2832672396359959,
 1.0147378859525562,
 0.035,
 0.2117108367626886,
 0.15992735042735043,
 0.

In [196]:
lista_dias_atraso_total

[15116.0,
 1.0,
 1.0,
 243.0,
 0.0,
 0.0,
 332.0,
 1.0,
 7240.0,
 90.0,
 323.0,
 13376.0,
 1018.0,
 4228.0,
 6028.0,
 6790.0,
 0.0,
 15.0,
 5025.0,
 1.0,
 1731.0,
 3.0,
 5930.0,
 0.0,
 2231.0,
 224.0,
 9.0,
 57.0,
 3.0,
 1.0,
 0.0,
 6291.0,
 0.0,
 30.0,
 2807.0,
 0.0,
 2.0,
 998.0,
 0.0,
 9354.0,
 0.0,
 2345.0,
 3.0,
 4.0,
 1385.0,
 25.0,
 268.0,
 1642.0,
 209.0,
 0.0,
 11124.0,
 88.0,
 4.0,
 568.0,
 5.0,
 6593.0,
 17.0,
 5.0,
 4.0,
 0.0,
 10.0,
 6661.0,
 0.0,
 9057.0,
 312.0,
 540.0,
 315.0,
 1306.0,
 10.0,
 41.0,
 631.0,
 32.0,
 528.0,
 13.0,
 21.0,
 0.0,
 52.0,
 2.0,
 2.0,
 12924.0,
 4068.0,
 131.0,
 6.0,
 2.0,
 209.0,
 90.0,
 7.0,
 0.0,
 0.0,
 34.0,
 390.0,
 8212.0,
 6.0,
 5.0,
 33.0,
 42.0,
 18689.0,
 183.0,
 2837.0,
 4.0,
 5.0,
 0.0,
 212.0,
 0.0,
 1164.0,
 8.0,
 27.0,
 12687.0,
 19.0,
 7.0,
 13494.0,
 38.0,
 9.0,
 3.0,
 198.0,
 11468.0,
 0.0,
 8.0,
 41.0,
 2317.0,
 0.0,
 83.0,
 592.0,
 0.0,
 58.0,
 0.0,
 227.0,
 17.0,
 3105.0,
 4.0,
 1.0,
 5.0,
 2.0,
 0.0,
 7.0,
 6.0,
 35.0,
 34

In [197]:
lista_parcelas_atraso

[23,
 1,
 1,
 8,
 0,
 0,
 7,
 1,
 13,
 4,
 7,
 23,
 10,
 16,
 20,
 15,
 0,
 2,
 15,
 1,
 19,
 3,
 24,
 0,
 22,
 6,
 2,
 12,
 1,
 1,
 0,
 19,
 0,
 8,
 14,
 0,
 1,
 12,
 0,
 23,
 0,
 5,
 2,
 1,
 15,
 4,
 10,
 14,
 9,
 0,
 13,
 6,
 1,
 12,
 2,
 13,
 5,
 2,
 1,
 0,
 4,
 15,
 0,
 14,
 9,
 14,
 8,
 10,
 2,
 4,
 8,
 1,
 14,
 3,
 1,
 0,
 5,
 1,
 1,
 13,
 13,
 9,
 5,
 1,
 6,
 4,
 5,
 0,
 0,
 3,
 7,
 13,
 4,
 4,
 4,
 7,
 13,
 12,
 10,
 3,
 2,
 0,
 9,
 0,
 12,
 2,
 5,
 14,
 6,
 3,
 13,
 5,
 2,
 3,
 4,
 13,
 0,
 4,
 2,
 20,
 0,
 11,
 8,
 0,
 6,
 0,
 11,
 3,
 14,
 3,
 1,
 2,
 1,
 0,
 4,
 2,
 7,
 13,
 7,
 7,
 9,
 6,
 8,
 0,
 0,
 0,
 2,
 17,
 9,
 13,
 22,
 8,
 1,
 3,
 0,
 6,
 0,
 12,
 12,
 0,
 0,
 0,
 0,
 10,
 4,
 5,
 0,
 5,
 6,
 14,
 7,
 0,
 5,
 1,
 1,
 3,
 0,
 9,
 1,
 4,
 5,
 11,
 7,
 19,
 3,
 9,
 0,
 2,
 20,
 0,
 4,
 9,
 0,
 4,
 20,
 0,
 2,
 2,
 20,
 0,
 2,
 0,
 4,
 13,
 22,
 14,
 3,
 4,
 3,
 3,
 5,
 3,
 0,
 14,
 6,
 22,
 5,
 0,
 17,
 1,
 2,
 1,
 9,
 1,
 2,
 0,
 7,
 4,
 16,
 9,
 1,
 10,
 12,
 0,
 

In [198]:
lista_parcelas

[22,
 8,
 13,
 32,
 1,
 5,
 13,
 13,
 14,
 8,
 13,
 22,
 22,
 17,
 21,
 14,
 6,
 2,
 14,
 3,
 21,
 14,
 24,
 1,
 22,
 12,
 13,
 14,
 2,
 12,
 8,
 19,
 1,
 13,
 14,
 7,
 5,
 13,
 12,
 22,
 11,
 14,
 3,
 2,
 21,
 13,
 14,
 14,
 22,
 7,
 13,
 12,
 7,
 11,
 14,
 14,
 12,
 14,
 11,
 1,
 12,
 14,
 14,
 14,
 13,
 14,
 14,
 19,
 12,
 23,
 15,
 7,
 19,
 14,
 13,
 1,
 10,
 5,
 13,
 14,
 13,
 14,
 14,
 1,
 14,
 5,
 11,
 2,
 2,
 5,
 14,
 12,
 10,
 17,
 12,
 14,
 14,
 14,
 12,
 13,
 12,
 4,
 35,
 1,
 16,
 3,
 12,
 14,
 14,
 9,
 14,
 14,
 14,
 13,
 10,
 13,
 2,
 12,
 12,
 22,
 0,
 13,
 9,
 33,
 27,
 2,
 14,
 4,
 14,
 12,
 14,
 12,
 6,
 2,
 12,
 13,
 12,
 14,
 14,
 12,
 11,
 12,
 22,
 2,
 4,
 4,
 12,
 22,
 14,
 14,
 23,
 13,
 3,
 14,
 1,
 14,
 2,
 14,
 26,
 12,
 5,
 6,
 11,
 12,
 14,
 12,
 9,
 14,
 13,
 14,
 22,
 2,
 14,
 2,
 14,
 12,
 0,
 10,
 4,
 9,
 12,
 14,
 21,
 24,
 16,
 10,
 2,
 12,
 22,
 2,
 6,
 12,
 13,
 6,
 25,
 1,
 3,
 3,
 26,
 12,
 14,
 2,
 13,
 14,
 22,
 14,
 13,
 15,
 13,
 14,
 12,
 14,

In [199]:
# for i in range (len(lista_parcelas)):
#     if(lista_parcelas[i] == 0):
#         lista_parcelas[i] = 1


In [202]:
lista_razao_parcelas_atraso_parcelas = []
for i in range(len(lista_parcelas)):
    if(lista_parcelas[i] == 0):
        lista_razao_parcelas_atraso_parcelas.append(0)
    else:
        lista_razao_parcelas_atraso_parcelas.append(lista_parcelas_atraso[i] / lista_parcelas[i])
lista_razao_parcelas_atraso_parcelas

[1.0454545454545454,
 0.125,
 0.07692307692307693,
 0.25,
 0.0,
 0.0,
 0.5384615384615384,
 0.07692307692307693,
 0.9285714285714286,
 0.5,
 0.5384615384615384,
 1.0454545454545454,
 0.45454545454545453,
 0.9411764705882353,
 0.9523809523809523,
 1.0714285714285714,
 0.0,
 1.0,
 1.0714285714285714,
 0.3333333333333333,
 0.9047619047619048,
 0.21428571428571427,
 1.0,
 0.0,
 1.0,
 0.5,
 0.15384615384615385,
 0.8571428571428571,
 0.5,
 0.08333333333333333,
 0.0,
 1.0,
 0.0,
 0.6153846153846154,
 1.0,
 0.0,
 0.2,
 0.9230769230769231,
 0.0,
 1.0454545454545454,
 0.0,
 0.35714285714285715,
 0.6666666666666666,
 0.5,
 0.7142857142857143,
 0.3076923076923077,
 0.7142857142857143,
 1.0,
 0.4090909090909091,
 0.0,
 1.0,
 0.5,
 0.14285714285714285,
 1.0909090909090908,
 0.14285714285714285,
 0.9285714285714286,
 0.4166666666666667,
 0.14285714285714285,
 0.09090909090909091,
 0.0,
 0.3333333333333333,
 1.0714285714285714,
 0.0,
 1.0,
 0.6923076923076923,
 1.0,
 0.5714285714285714,
 0.52631578947

In [203]:
dataFrame['razao_pago_total'] = lista_razao_pago_total
dataFrame['dias_atraso_total'] = lista_dias_atraso_total
dataFrame['razao_parcelas_atraso_parcelas'] = lista_razao_parcelas_atraso_parcelas

In [204]:
dataFrame

,id_contrato,sexo,estado_civil,tipo,inadimplente,status,tier,valor_contrato,score,idade,razao_pago_total,dias_atraso_total,razao_parcelas_atraso_parcelas
0,1324.0,2.0,1,1,1,2,6,88683.53,9999.00,36,0.000000,15116.0,1.045455
1,4173.0,1.0,2,1,1,1,3,195900.00,995.83,33,0.166694,1.0,0.125000
2,2125.0,2.0,2,1,1,1,4,65900.00,1000.00,35,0.178498,1.0,0.076923
3,335.0,1.0,3,1,1,1,5,64592.88,757.11,35,0.413484,243.0,0.250000
4,4544.0,1.0,2,1,1,1,5,74900.00,1000.00,30,0.055620,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3958,739.0,1.0,2,1,1,2,6,65310.11,9999.00,43,0.035000,16298.0,1.000000
3959,1655.0,1.0,1,1,1,2,6,72900.00,9999.00,26,0.120881,2675.0,0.846154
3960,4161.0,1.0,1,1,2,1,6,67900.00,311.11,41,0.053284,713.0,0.888889
3961,4347.0,1.0,1,1,1,1,5,67900.00,787.04,26,0.075008,126.0,0.714286


In [ ]:
dataFrame.to_csv('infos.csv', index=False)
dataFrame.to_excel('infos.xlsx', index=False)